In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsFeature(df)
    df = ATRFeature(df)
    df = KeltnerChannelsFeature(df)
    df = RSIFeature(df)
    df = MACDFeature(df)
    df['pivots_l'] = False
    df['pivots_h'] = False

    df['low_perv'] = df.low.shift(1)
    df['high_perv'] = df.high.shift(1)
    df['low_next'] = df.low.shift(-1)
    df['high_next'] = df.high.shift(-1)
    df['pivots_l'] = (df.low_perv > df.low) & (df.low_next > df.low)
    df['pivots_h'] = (df.high_perv < df.high) & (df.high_next < df.high)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df.pivots_l = df.pivots_l.astype(float)
    df.pivots_h = df.pivots_h.astype(float)
    return df

In [3]:
BUY = 2
SELL = 0
NONE = 1

In [4]:
def labeling(df):
    df['SIGNAL_LABEL'] = NONE
    df['TP'] = 0
    df['SL'] = 0
    
    for i in range(1,len(df)):
        if df.pivots_l[i-1]:
            for j in range(i,len(df)):
                if( df.low[j] <= df.low[i-1] ):
                    break
                else:
                    if(df.high[j] > 2*(df.close[i] - df.low[i-1] ) + df.close[i]):
                        df.SIGNAL_LABEL[i] = BUY
                        break

            df['SL'][i] = df.low[i-1] 
            df['TP'][i] = 2*(df.close[i] - df.low[i-1] ) + df.close[i]
        if df.pivots_h[i-1]:
            for j in range(i,len(df)):
                if( df.high[j] >= df.high[i-1] ):
                    break
                else:
                    if(df.low[j] < 2*(df.close[i] - df.high[i-1] ) + df.close[i]):
                        df.SIGNAL_LABEL[i] = SELL
                        break
            df['SL'][i] = df.high[i-1] 
            df['TP'][i] = 2*(df.close[i] - df.high[i-1] ) + df.close[i]
        
    return df

In [5]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [6]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

9.000000000014552e-05

In [7]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

9e-05

In [8]:
SLIT_INDEX = int(0.8*len(df_an))
df_train = df_an.iloc[:SLIT_INDEX]
df_train.reset_index(drop=True, inplace=True)
df_test = df_an.iloc[SLIT_INDEX:]
df_test.reset_index(drop=True, inplace=True)

In [9]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [10]:
df_train = featureExtraction(df_train)
df_train = labeling(df_train)
df_test = featureExtraction(df_test)
df_test = labeling(df_test)


C:\Users\meysam\AppData\Local\Temp\ipykernel_9984\757576024.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SL'][i] = df.high[i-1]
C:\Users\meysam\AppData\Local\Temp\ipykernel_9984\757576024.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TP'][i] = 2*(df.close[i] - df.high[i-1] ) + df.close[i]
C:\Users\meysam\AppData\Local\Temp\ipykernel_9984\757576024.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.SIGNAL_LAB

In [11]:
len(df_test)

3966

In [12]:
feature_col = [
    'open', 'high', 'low', 'close',
    
    'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c',
    'Feat_EMA_c', 'Feat_KeUp_c', 'Feat_KeLo_c',
    
    'Feat_BB_MA_o', 'Feat_BB_UP_o', 'Feat_BB_LW_o',
    'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 
    
    'Feat_BB_MA_l', 'Feat_BB_UP_l', 'Feat_BB_LW_l',
    'Feat_EMA_l', 'Feat_KeUp_l', 'Feat_KeLo_l',
    
    'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
    'Feat_EMA_h', 'Feat_KeUp_h', 'Feat_KeLo_h',
    
    'Feat_ATR_14',
    'Feat_gains','Feat_wins_rma', 'Feat_losses_rma', 'Feat_RSI_14', 
#     'Feat_MACD', 'Feat_SIGNAL_MACD', 'Feat_HIST',
    
        'full_range',# this is added ...
       'body_lower', 'body_upper', 
        'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
    'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [13]:
class RNN(nn.Module):
    def __init__(self, batch_size, input_size, hidden_dim,lstm_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.LSTM_layers = lstm_layers
    
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=self.hidden_dim, num_layers=self.LSTM_layers, batch_first=True)
        self.fc = nn.Linear(in_features=self.hidden_dim, out_features=3)
#         self.fc2 = nn.Linear(257, 1)

    def forward(self, x):
        h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
        c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
        torch.nn.init.xavier_normal_(h)
        torch.nn.init.xavier_normal_(c)
        out, (hidden, cell) = self.lstm(x, (h,c))
#         out = self.dropout(out)
        out = self.fc(out[:,-1,:]) 
#         out = torch.sigmoid(self.fc2(out))
        return out

In [16]:
class dataset(Dataset):
    def __init__(self,df,sequence_lenght,feature_col):
        self.df = df
        self.sequence_lenght = sequence_lenght
        self.feature_col = feature_col
        self.X = []
        self.Y = [] 
        self.indeces = []
        print("preparing data..")
        self.prepare_data()
        
    def prepare_data(self):
        for i in range(self.sequence_lenght+1,len(self.df)):
            if self.df.pivots_h[i-1] or self.df.pivots_l[i-1]:
                X_feat = [] 
                for j in range(self.sequence_lenght):
                    feature = []
                    feature.append(self.df.pivots_h[i-self.sequence_lenght+j])
                    feature.append(self.df.pivots_l[i-self.sequence_lenght+j])
                    for fc in self.feature_col:
                        feature.append(self.df[fc][i-self.sequence_lenght+j+1])
                        
                    X_feat.append(feature)  
                self.X.append(np.array(X_feat))# TODO: use torch tensor 
                self.Y.append(self.df.SIGNAL_LABEL[i])
                self.indeces.append(i)
                
    def get_feature_len(self):
        return self.X[0].shape[1]
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self,index):
        return torch.from_numpy(self.X[index]).float(),self.Y[index]

In [17]:
epoch = 1000
batch_size = 600
sequence_lenght = 100

In [18]:
ds_train = dataset(df_train,sequence_lenght,feature_col)
ds_test = dataset(df_test,sequence_lenght,feature_col) 

loader_train = DataLoader(ds_train, batch_size=batch_size)
loader_test = DataLoader(ds_test, batch_size=batch_size)

preparing data..
preparing data..


In [19]:
print(len(loader_train))
print(len(loader_test))
print(len(ds_train))
print(len(ds_test))

12
3
6860
1690


In [20]:
input_size = ds_train.get_feature_len() 
hidden_dim = 60
lstm_layers = 2

In [21]:
model = RNN(batch_size, input_size, hidden_dim,lstm_layers)

In [22]:
weight = torch.Tensor([0.4,0.2,0.4])

In [23]:
criterion = nn.CrossEntropyLoss(weight=weight)

In [24]:
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [25]:
model.train()
train_loss = []
test_loss = []
test_accuracy = []
for i in range(epoch):
    loss_epoch_train = 0
    for x_batch, y_batch in loader_train:
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch_train += loss.detach().item()
        
    
    loss_epoch_test = 0
    model.eval()
    pred_test = []
    all_label = []
    
    for x_batch, y_batch in loader_test:
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        loss_epoch_test += loss.detach().item()
        pred_test += list(pred.softmax(dim=1).argmax(dim=1).detach().numpy())
        all_label += list(y_batch.detach().numpy())
        
    train_loss.append(loss_epoch_train/len(loader_train))
    test_loss.append(loss_epoch_test/len(loader_test))
    pred_test = np.array(pred_test)
    all_label = np.array(all_label)
    test_accuracy.append((pred_test==all_label).sum()/len(all_label))
    print(train_loss[-1],test_accuracy[-1])

1.0889968574047089 0.6
1.078487515449524 0.6
1.070744256178538 0.6
1.0568132400512695 0.5970414201183432
1.0235792150100071 0.5556213017751479
0.9527058998743693 0.4863905325443787
0.8801776667435964 0.45384615384615384
0.8205914547046026 0.4437869822485207
0.7788264701763788 0.4319526627218935
0.7592440545558929 0.4372781065088757
0.7498241563638052 0.4331360946745562
0.7427043318748474 0.4349112426035503
0.7379875729481379 0.43372781065088756
0.7358802060286204 0.44733727810650886
0.7340131203333536 0.4396449704142012
0.7321237872044245 0.4437869822485207
0.7302048802375793 0.4349112426035503
0.7291756669680277 0.4502958579881657
0.7279917945464452 0.4408284023668639
0.7267005443572998 0.45798816568047335
0.7255240976810455 0.4650887573964497
0.7244363725185394 0.4556213017751479
0.722898930311203 0.44970414201183434
0.7214390337467194 0.4437869822485207
0.7200078219175339 0.4467455621301775
0.7186364829540253 0.44497041420118344
0.7173129270474116 0.4532544378698225
0.71620641648769

0.03894387767650187 0.5284023668639053
0.03689646457011501 0.5301775147928994
0.03652586632718643 0.5384615384615384
0.03555926331318915 0.5301775147928994
0.0343068679018567 0.5384615384615384
0.033677061165993415 0.5349112426035503
0.035662546986714005 0.5337278106508876
0.0328370724649479 0.5360946745562131
0.03377493044051031 0.5260355029585799
0.03848291599812607 0.5414201183431953
0.044251559457431235 0.5396449704142012
0.050923618177572884 0.5331360946745562
0.05526078213006258 0.5402366863905326
0.07130432361736894 0.5349112426035503
0.10237380241354306 0.5337278106508876
0.12947590090334415 0.5520710059171597
0.13795611634850502 0.5443786982248521
0.15210479063292345 0.5207100591715976
0.13204275816679 0.5396449704142012
0.13252702013899884 0.5313609467455621
0.1197121621419986 0.5467455621301776
0.09667598564798634 0.5502958579881657
0.07512759106854598 0.5284023668639053
0.05800904675076405 0.5384615384615384
0.04034591435144345 0.5349112426035503
0.032813334527115025 0.5378

KeyboardInterrupt: 

In [ ]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
#         df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_l[idx-1]==1 and y_perdict[i] ==BUY:
            df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_h[idx-1]==1 and y_perdict[i] ==SELL:
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio =1# abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = 2#abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])